## Setup

Run the cells below to setup and install the required libraries. For our experiment we will need `accelerate`, `peft`, `transformers`, `datasets` and TRL to leverage the recent [`SFTTrainer`](https://huggingface.co/docs/trl/main/en/sft_trainer). We will use `bitsandbytes` to [quantize the base model into 4bit](https://huggingface.co/blog/4bit-transformers-bitsandbytes). We will also install `einops` as it is a requirement to load Falcon models.

In [ ]:
%pip install  -U trl transformers accelerate git+https://github.com/huggingface/peft.git
%pip install  datasets bitsandbytes einops wandb

## Loading the model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from accelerate import infer_auto_device_map
model_name = "codellama/CodeLlama-13b-Instruct-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto",
    offload_folder='offload'
)

Let's also load the tokenizer below

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
example = {
    'question': 
        "Can you solve this real interview question? Find The Original Array of Prefix Xor - You are given an integer array pref of size n. Find and return the array arr of size n that satisfies:\n\n * pref[i] = arr[0] ^ arr[1] ^ ... ^ arr[i].\n\nNote that ^ denotes the bitwise-xor operation.\n\nIt can be proven that the answer is unique.\n\n \n\nExample 1:\n\n\nInput: pref = [5,2,0,3,1]\nOutput: [5,7,2,3,2]\nExplanation: From the array [5,7,2,3,2] we have the following:\n- pref[0] = 5.\n- pref[1] = 5 ^ 7 = 2.\n- pref[2] = 5 ^ 7 ^ 2 = 0.\n- pref[3] = 5 ^ 7 ^ 2 ^ 3 = 3.\n- pref[4] = 5 ^ 7 ^ 2 ^ 3 ^ 2 = 1.\n\n\nExample 2:\n\n\nInput: pref = [13]\nOutput: [13]\nExplanation: We have pref[0] = arr[0] = 13.\n\n\n \n\nConstraints:\n\n * 1 <= pref.length <= 105\n * 0 <= pref[i] <= 106",
    "code": "class Solution {\npublic:\n    vector<int> findArray(vector<int>& pref) {\n        int n = pref.size();\n        \n        vector<int> arr;\n        arr.push_back(pref[0]);\n        for (int i = 1; i < n; i++) {\n            arr.push_back(pref[i] ^ pref[i - 1]);\n        }\n        \n        return arr;\n    }\n};"
}  # Replace 0 with the index of the example you want to use
text = f"""
    <s>[INST] <<SYS>>
    {{You are a smart AI Explainer. You will explain what is happening in the ###SOLUTION based on what is asked in the ###QUESTION and add the explaination after the ###EXPLANATION }}
    <</SYS>>
    ###QUESTION: {example['question']} 
    ###SOLUTION: {example['code']}[/INST]
    
    ###EXPLANATION:"""
text

In [ ]:
device = "cuda:0"

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt").to(device)

# Generate the output using the model
outputs = model.generate(**inputs, max_new_tokens=200)

# Decode and print the output
print(tokenizer.decode(outputs[0], skip_special_tokens=True))